In [55]:
import string
import csv

def clean_string(s):
    """
    Cleans column names from the features.txt file.
    Rules for cleaning column names: 
    1. Remove parentheses
    2. Remove dashes
    3. Replace commas with underscores
    """

    translations = string.maketrans(",", "_")
    deletions = "()-"
    return s.translate(translations, deletions)

def clean_column_names(fname_in, fname_out):
    """
    Cleans column names and writes to output file 
    """
    SPACE = " "
    with open(fname_in, "rb") as f_in, open(fname_out, "wb") as f_out:
        reader = csv.reader(f_in, delimiter=SPACE)
        writer = csv.writer(f_out, delimiter=SPACE)
        for line in reader:
            writer.writerow([line[0], clean_string(line[1])])

def main():
    """
    Driver
    """ 
    in_file = r'.\features.txt'
    out_file = r'.\features_clean.txt'
    clean_column_names(in_file, out_file)

if __name__ == '__main__':
    main()

In [57]:
import csv

X_train = open(r'C:\Users\Russell Pekrul\Downloads\UCI HAR Dataset\UCI HAR Dataset\train\X_train.txt', "rb").read().splitlines()
y_train = open(r'C:\Users\Russell Pekrul\Downloads\UCI HAR Dataset\UCI HAR Dataset\train\y_train.txt', "rb").read().splitlines()
subject_train = open(r'C:\Users\Russell Pekrul\Downloads\UCI HAR Dataset\UCI HAR Dataset\train\subject_train.txt', "rb").read().splitlines()

X_test = open(r'C:\Users\Russell Pekrul\Downloads\UCI HAR Dataset\UCI HAR Dataset\test\X_test.txt', "rb").read().splitlines()
y_test = open(r'C:\Users\Russell Pekrul\Downloads\UCI HAR Dataset\UCI HAR Dataset\test\y_test.txt', "rb").read().splitlines()
subject_test = open(r'C:\Users\Russell Pekrul\Downloads\UCI HAR Dataset\UCI HAR Dataset\test\subject_test.txt', "rb").read().splitlines()

X_all = X_train + X_test
y_all = y_train + y_test
subject_all = subject_train + subject_test

with    open(r'.\X_all.txt', "wb") as Xf, \
            open(r'.\y_all.txt', "wb") as yf, \
            open(r'.\subject_all.txt', "wb") as sf:
        X_writer = csv.writer(Xf)
        y_writer = csv.writer(yf)
        subject_writer = csv.writer(sf)

        for xrow, yrow, srow in zip(X_all, y_all, subject_all):
            X_writer.writerow([xrow])
            y_writer.writerow([yrow])
            subject_writer.writerow([srow])

In [58]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_score

# read feature names
feature_names = open(r'.\features_clean.txt').read().splitlines()
col_names = [f.split(" ")[1] for f in feature_names]

# Create data frames from combined data
df_X = pd.read_csv(r'.\X_all.txt', sep=" *", engine="python", header=None, names=col_names)
df_y = pd.read_csv(r'.\y_all.txt', header=None, names=["activity"])
df_sub = pd.read_csv(r'.\subject_all.txt', header=None, names=["subject"])

# Choose training, testing and validation data Scikit-learn doesnt natively
# support pandas. So Convert training and testing sets into matrices  for
# feeding to classifier algorithms

training_set = df_sub["subject"] >= 27
test_set = df_sub["subject"] <= 6
cv_set = (df_sub["subject"] >= 21) & (df_sub["subject"] < 27)

X_train = df_X[training_set].as_matrix()
y_train = df_y[training_set].as_matrix().squeeze()

X_test = df_X[test_set].as_matrix()
y_test = df_y[test_set].as_matrix().squeeze()

X_cv = df_X[cv_set].as_matrix()
y_cv = df_y[cv_set].as_matrix().squeeze()

clf = RandomForestClassifier(n_estimators=50, oob_score=True)
clf.fit(X_train, y_train)
print clf.oob_score_

0.988552188552
